<a href="https://colab.research.google.com/github/JgazteluML/STM32Cube_FW_F4/blob/master/giroscopio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import requests
import datetime
from tensorboard.plugins.hparams import api as hp

API_KEY = 'ei_7640d6d5870987290c6670e19996c09dde1e16a3382ba358f2463e1a53312fa1'

def download_data(url):
    response = requests.get(url, headers={'x-api-key': API_KEY})
    if response.status_code == 200:
        return response.content
    else:
        print(response.content)
        raise ConnectionError('Could not download data file')

X = download_data('https://studio.edgeimpulse.com/v1/api/113048/training/65/x')
Y = download_data('https://studio.edgeimpulse.com/v1/api/113048/training/65/y')
with open('x_train.npy', 'wb') as file:
    file.write(X)
with open('y_train.npy', 'wb') as file:
    file.write(Y)
X = np.load('x_train.npy')
Y = np.load('y_train.npy')[:,0]
import sys, os, random
import tensorflow as tf
from sklearn.model_selection import train_test_split

import logging
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set random seeds for repeatable results
RANDOM_SEED = 3
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

classes_values = [ "Horizontal", "agitar_ArribaAbajo", "agitar_IzqDer" ]
classes = len(classes_values)

Y = tf.keras.utils.to_categorical(Y - 1, classes)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

input_length = X_train[0].shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

def get_reshape_function(reshape_to):
    def reshape(image, label):
        return tf.reshape(image, reshape_to), label
    return reshape

callbacks = []

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Conv1D, Conv2D, Flatten, Reshape, MaxPooling1D, MaxPooling2D, BatchNormalization, TimeDistributed, ReLU, Softmax
from tensorflow.keras.optimizers import Adam
EPOCHS = 60
# this controls the batch size, or you can manipulate the tf.data.Dataset objects yourself
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=False)
validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=False)

# model architecture
model = Sequential()
model.add(Dense(30, activation='relu',
    activity_regularizer=tf.keras.regularizers.l1(0.00001)))
model.add(Dropout(0.2))
model.add(Dense(30, activation='relu',
    activity_regularizer=tf.keras.regularizers.l1(0.00001)))
model.add(Dropout(0.2))
model.add(Dense(classes, name='y_pred', activation='softmax'))

# this controls the learning rate
opt = Adam(learning_rate=0.004, beta_1=0.9, beta_2=0.999)



log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)
hparams_callback = hp.KerasCallback(log_dir, {
    'num_relu_units': 512,
    'dropout': 0.2
})


# train the neural network
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(train_dataset, epochs=EPOCHS, validation_data=validation_dataset, verbose=2, callbacks=[tensorboard_callback, hparams_callback])
#          .get_class_weights(Y_train))

# Use this flag to disable per-channel quantization for a model.
# This can reduce RAM usage for convolutional models, but may have
# an impact on accuracy.
disable_per_channel_quantization = False
# Save the model to disk
model.save('saved_model')

!tensorboard dev upload --logdir ./logs \
  --name "giroscope" \
  --description "Training results from https://colab.research.google.com/drive/1gLFeZ8u3Hh-eTY2BzacgVbAlMRKyC79g#scrollTo=MJk4Wxm7T232 \
  --one_shot

Epoch 1/60
5/5 - 1s - loss: 37803.0703 - accuracy: 0.3256 - val_loss: 1978.5164 - val_accuracy: 0.4848 - 791ms/epoch - 158ms/step
Epoch 2/60
5/5 - 0s - loss: 9096.5234 - accuracy: 0.4729 - val_loss: 4336.3828 - val_accuracy: 0.4545 - 64ms/epoch - 13ms/step
Epoch 3/60
5/5 - 0s - loss: 6801.3262 - accuracy: 0.5116 - val_loss: 799.0781 - val_accuracy: 0.5152 - 55ms/epoch - 11ms/step
Epoch 4/60
5/5 - 0s - loss: 2802.1152 - accuracy: 0.5969 - val_loss: 398.7759 - val_accuracy: 0.6061 - 51ms/epoch - 10ms/step
Epoch 5/60
5/5 - 0s - loss: 5501.4487 - accuracy: 0.5271 - val_loss: 257.9020 - val_accuracy: 0.6061 - 50ms/epoch - 10ms/step
Epoch 6/60
5/5 - 0s - loss: 2695.3662 - accuracy: 0.6512 - val_loss: 131.2227 - val_accuracy: 0.6061 - 48ms/epoch - 10ms/step
Epoch 7/60
5/5 - 0s - loss: 709.9233 - accuracy: 0.6899 - val_loss: 46.3459 - val_accuracy: 0.6970 - 50ms/epoch - 10ms/step
Epoch 8/60
5/5 - 0s - loss: 590.1388 - accuracy: 0.7597 - val_loss: 16.1773 - val_accuracy: 0.8485 - 53ms/epoch - 1